## REQUIRED PACKAGES

In [ ]:
# Machine learning packages
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Reshape

# Dataset management packages
from spivutils.synthetic_datasets.spid import load_data
from spivutils.batch_generators.keras_generator import batch_data
from spivutils.common_tools.operations import normalization, vectoraddition, thresholding, imagecropping

# 
from numpy import prod, zeros

#
import pandas as pd

#
import json

## HARDWARE SETTINGS

In [ ]:
# Detects the avaiable hardware
cpu_devices = tf.config.list_physical_devices('CPU')
gpu_devices = tf.config.experimental.list_physical_devices('GPU')

# Allow memory growth
for gpu in gpu_devices:
  tf.config.experimental.set_memory_growth(gpu, True)

## MODEL HYPERPARAMETERS

In [ ]:
chunk_size = 100#15119
batch_size = 5
num_epoch  = 10

filters = 32
units = [32, 64, 128, 256, 512, 1024, 2048]
#kernel_size = (1,64,64)
#pool_size = (1,3,3)

filters = [32, 64, 128, 256, 512, 1024, 2048]
kernel_size = [(1,3,3), (1,5,5), (1,7,7), (1,9,9), (1,11,11)]
pool_size = [(1,2,2), (1,4,4), (1,6,6), (1,8,8), (1,10,10)]

activation_function = 'LeakyReLU'

optimizer_function = 'adam'                                      # Weight update after each iteration
loss_function      = 'mean_squared_error'                             # Sets the Loss function

## DATA LOADING

In [ ]:
# Imports train and validation data
(train_x, train_y), (valid_x, valid_y), _ = load_data()

# Collect a chunk of the dataset
train_x_chunk = train_x[0:int(chunk_size*0.7)]
train_y_chunk = train_y[0:int(chunk_size*0.7)]
valid_x_chunk = valid_x[0:int(chunk_size*0.15)]
valid_y_chunk = valid_y[0:int(chunk_size*0.15)]

# Load data in batches to avoid memory overload
train_batch = batch_data(train_x_chunk, train_y_chunk, batch_size)
valid_batch = batch_data(valid_x_chunk, valid_y_chunk, batch_size)

## DATA PREPROCESSING

In [ ]:

def standardization(input_data):

    output_data = zeros(input_data.shape)

    #min = np.min(input_data)
    #max = np.max(input_data)

    max = 8.75
    min = 0

    output_data[:, :, :] = (input_data[:, :, :] - min)/(max - min)

    #for i in range(2):

    #    output_data[:, i, :, :] = (input_data[:, i, :, :] - min)/(max - min)

    return output_data

In [ ]:


# Crops a square region with size equals to small image dimension at the center of the image
train_batch.add_x_preprocessing_operation(imagecropping)
valid_batch.add_x_preprocessing_operation(imagecropping)

# Crops a square region with size equals to small image dimension at the center of the image
train_batch.add_y_preprocessing_operation(imagecropping)
valid_batch.add_y_preprocessing_operation(imagecropping)

#
train_batch.add_y_preprocessing_operation(vectoraddition)
valid_batch.add_y_preprocessing_operation(vectoraddition)

# Apply normalization to the input data
train_batch.add_x_preprocessing_operation(normalization)
valid_batch.add_x_preprocessing_operation(normalization)

# Apply normalization to the output data
train_batch.add_y_preprocessing_operation(standardization)
valid_batch.add_y_preprocessing_operation(standardization)

## MODEL DEFINITION

In [ ]:


input_shape = train_batch[0][0][0,].shape
output_shape = train_batch[0][1][0,].shape

model = Sequential()

model.add(Conv3D(filters = filters[0], kernel_size = kernel_size[4], activation = activation_function, input_shape = input_shape))
model.add(MaxPooling3D(pool_size = pool_size[4]))

model.add(Conv3D(filters = filters[1], kernel_size = kernel_size[2], activation = activation_function, input_shape = input_shape))
model.add(MaxPooling3D(pool_size = pool_size[2]))

model.add(Conv3D(filters = filters[2], kernel_size = kernel_size[1], activation = activation_function, input_shape = input_shape))
model.add(MaxPooling3D(pool_size = pool_size[1]))

#model.add(Conv3D(filters = filters[3], kernel_size = kernel_size[0], activation = activation_function, input_shape = input_shape))
#model.add(MaxPooling3D(pool_size = pool_size[0]))

model.add(Flatten())

model.add(Dense(units = units[0], activation = activation_function))

model.add(Dense(units = units[1], activation = activation_function))

#model.add(Dense(units = units[2], activation = activation_function))

#model.add(Dense(units = units[3], activation = activation_function))

#model.add(Dense(units = units[4], activation = activation_function))

model.add(Dense(units = units[3], activation = activation_function))

model.add(Dense(prod(output_shape)))

model.add(Reshape(output_shape))

model.compile(loss = loss_function, optimizer = optimizer_function)

## MODEL SUMMARIZATION

In [ ]:
model.summary()

## MODEL TRAINING

In [ ]:
history = model.fit(train_batch, validation_data = valid_batch, epochs = num_epoch)

In [ ]:
# Saves model parameters and weights 
model.save('model_1_23_00.keras')

# Saves the model training history
#history_df = pd.DataFrame(history.history)
#history_df.to_csv('training_history.csv', index = False)

with open('model_1_23_00_training_history.json', 'w') as file:
    json.dump(history.history, file)